<a href="https://colab.research.google.com/github/soares-f/JMeSH/blob/main/Inverted_index_from_LitCovid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Extracting LitCovid in local env




In [1]:
!wget https://ftp.ncbi.nlm.nih.gov/pub/lu/LitCovid/litcovid2pubtator.xml.gz

--2021-02-01 09:36:29--  https://ftp.ncbi.nlm.nih.gov/pub/lu/LitCovid/litcovid2pubtator.xml.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 2607:f220:41e:250::13, 2607:f220:41e:250::11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 588582956 (561M) [application/x-gzip]
Saving to: ‘litcovid2pubtator.xml.gz’

litcovid2pubtator.x 100%[===================>] 561.32M  34.4MB/s    in 17s     

2021-02-01 09:36:46 (33.1 MB/s) - ‘litcovid2pubtator.xml.gz’ saved [588582956/588582956]



In [2]:
!gunzip litcovid2pubtator.xml.gz

In [ ]:
!ls -lh

total 2.8G
drwx------ 5 root root 4.0K Jan 19 21:37 drive
-rw-r--r-- 1 root root 2.8G Jan 19 19:32 litcovid2pubtator.xml
drwxr-xr-x 1 root root 4.0K Jan 15 17:28 sample_data


In [3]:
pip install bioc

     |████████████████████████████████| 552kB 13.7MB/s 
     |████████████████████████████████| 5.8MB 14.4MB/s 
  Created wheel for bioc: filename=bioc-1.3.6-cp36-none-any.whl size=20269 sha256=42314fef4022a694b7607e40551a3d9cfd8d0ccbf71655e2f0f5a5d9a9a06ffa
  Stored in directory: /root/.cache/pip/wheels/3f/91/3a/dca9e2d2f1deafa2c88ff05817448711d9262049e238813b9f
Successfully built bioc
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [4]:
filename = 'litcovid2pubtator.xml'

In [16]:
from nltk import tokenize
import nltk
nltk.download('punkt')
import collections
import uuid
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import bioc
import io
# read from a ByteIO
all_passages = []
with open(filename, 'rb') as fp:
  reader = bioc.BioCXMLDocumentReader(fp)
  collection_info = reader.get_collection_info()
  i = 0
  for document in reader:
    for passage in document.passages:
      tmp = process_passage(passage)
      if tmp is not None:
        all_passages.append(tmp)
      i+=1
      print(i)
      if i % 30000==0:
          break


In [76]:
process_passage(passage)

defaultdict(list,
            {'annotations': [{'id': '3',
               'identifier': 'MESH:D016638',
               'text': 'Critical illness'},
              {'id': '4', 'identifier': 'MESH:C000657245', 'text': 'COVID-19'},
              {'id': '5', 'identifier': 'MESH:D007239', 'text': 'infection'}],
             'initial_text': 'Critical illness-associated cerebral microbleeds in severe COVID-19 infection ',
             'initial_text_uuid': UUID('f590b85f-c712-4670-adbd-4104901e1723'),
             'new_text': '<tag id=MESH:D016638>Critical illness</tag>-associated cerebral microbleeds in severe <tag id=MESH:C000657245>COVID-19</tag> <tag id=MESH:D007239>infection</tag> '})

In [63]:
text = '<tag id=MESH:D016638>Critical illness</tag>-associated cerebral microbleeds in severe <tag id=MESH:C000657245>COVID-19</tag> <tag id=MESH:D007239>infection</tag>'


In [73]:
import requests
headers = {'Content-type': 'application/x-www-form-urlencoded'}
files = {
    'text' : [text,text],
    'auth_key': '37da81f1-24a5-3846-f387-46eccf79d761',
    'target_lang': 'JA',
    'tag_handling': 'xml',
    'split_sentences': 0
}

response2 = requests.post('https://api.deepl.com/v2/translate', data=files, headers = headers)

In [77]:
def process_passage(passage,type_of_annotation='MESH'):
  dict_passage = collections.defaultdict(list)
  initial_text = passage.text
  dict_passage['annotations'] = []
  for annotation in passage.annotations:
    tmp =  {k.lower(): v for k, v in annotation.infons.items()}
    if type_of_annotation in tmp['identifier']:
      annotation_text = annotation.text
      annotation_identifier = tmp['identifier']
      annotation_id = annotation.id
      dict_passage['annotations'].append({'text':annotation_text,
                                              'identifier': annotation_identifier,
                                              'id': annotation_id})
  if len(dict_passage['annotations']) > 0:
    new_text = initial_text
    for item in dict_passage['annotations']:
      new_text = re.sub(r"(?<!>){}".format(item['text']), f"<tag id={item['identifier']}>{item['text']}</tag>", new_text)
    dict_passage['initial_text'] = initial_text
    dict_passage['initial_text_uuid'] = str(uuid.uuid4())
    dict_passage['new_text'] = new_text
    return(dict_passage)

  else:
    return(None)

